In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 📌 1. 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
import gc
import os

In [3]:
# 📌 2. 메모리 최적화 함수 정의
def optimize_types(df):
    for col in df.select_dtypes(include=['float64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int64']).columns:
        df[col] = pd.to_numeric(df[col], downcast='integer')
    return df

In [4]:
# 📌 3. 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/train_final_98.csv')

In [5]:
test = pd.read_csv('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/test_final_98.csv')

In [6]:
if 'Unnamed: 0' in train.columns:
    train = train.drop(columns=['Unnamed: 0'])

In [7]:
# ID 컬럼 제거
if 'ID' in train.columns:
    train = train.drop(columns=['ID'])

In [8]:
# 📌 4. 메모리 최적화 적용
train = optimize_types(train)
gc.collect()

0

In [18]:
# 📌 5. 피처/타깃 분리
X = train.drop(columns=['Segment'])
y = train['Segment']

# ✅ X.columns 저장 (예측 시 컬럼 순서 일치용)
with open('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/X_columns.pkl', 'wb') as f:
    pickle.dump(X.columns.tolist(), f)

In [10]:
# 📌 6. object → category 변환
for col in X.select_dtypes(include='object').columns:
    X[col] = X[col].astype('category')

In [11]:
# 📌 7. 타깃 라벨 인코딩
le = LabelEncoder()
y_encoded = le.fit_transform(y)

In [20]:
print(le.classes_)

['A' 'B' 'C' 'D' 'E']


In [12]:
# 📌 8. 클래스 불균형 대응 가중치 계산
from collections import Counter
class_counts = Counter(y_encoded)
total = sum(class_counts.values())
weights = {cls: total / count for cls, count in class_counts.items()}
sample_weights = np.array([weights[c] for c in y_encoded])

In [13]:
# 📌 9. K-Fold 설정
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [14]:
import pickle
import gc

for fold, (train_idx, val_idx) in enumerate(skf.split(X, y_encoded)):
    print(f"\n🚀 Fold {fold+1} 시작")

    # 학습 데이터 분할
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]
    w_train = sample_weights[train_idx]

    # 모델 생성
    model = xgb.XGBClassifier(
        objective='multi:softprob',
        num_class=len(np.unique(y_encoded)),
        eval_metric='mlogloss',
        tree_method='gpu_hist',
        device='cuda',
        enable_categorical=True,
        random_state=42,
        verbosity=0
    )

    model.fit(X_train, y_train, sample_weight=w_train)

    # 평가
    preds_val = model.predict(X_val)
    f1 = f1_score(y_val, preds_val, average='micro')
    print(f"✅ Fold {fold+1} f1_micro: {f1:.4f}")

    # 중요도 저장
    importance_dict = model.get_booster().get_score(importance_type='gain')

    with open(f'/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/importance_fold{fold}.pkl', 'wb') as f:
        pickle.dump(importance_dict, f)

    with open(f'/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/f1_fold{fold}.txt', 'w') as f:
        f.write(str(f1))

    # 메모리 정리
    del model, X_train, X_val, y_train, y_val, w_train, preds_val, importance_dict
    gc.collect()


🚀 Fold 1 시작
✅ Fold 1 f1_micro: 0.8892

🚀 Fold 2 시작
✅ Fold 2 f1_micro: 0.8891

🚀 Fold 3 시작
✅ Fold 3 f1_micro: 0.8903

🚀 Fold 4 시작
✅ Fold 4 f1_micro: 0.8895

🚀 Fold 5 시작
✅ Fold 5 f1_micro: 0.8884


In [15]:
10 / 0

ZeroDivisionError: division by zero

In [16]:
fold = 2  # 가장 성능 좋은 Fold 기준 모델 저장(Fold 3 → index 2)

train_idx, val_idx = list(skf.split(X, y_encoded))[fold]
X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train = y_encoded[train_idx]
w_train = sample_weights[train_idx]

model = xgb.XGBClassifier(
    objective='multi:softprob',
    num_class=len(np.unique(y_encoded)),
    eval_metric='mlogloss',
    tree_method='gpu_hist',
    device='cuda',
    enable_categorical=True,
    random_state=42,
    verbosity=0
)

model.fit(X_train, y_train, sample_weight=w_train)

# 저장
with open('/content/drive/MyDrive/멋쟁이사자처럼_파이널프로젝트/98제거/final_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("✅ Fold 3 모델 저장 완료")

✅ Fold 3 모델 저장 완료


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [08:21:42] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
